In [1]:
# Code for the computation of F_p solutions in the Albanese graph Alb_{p,1}
# Authored by: Philip Engel, Olivier de Gaay Fortman, and Stefan Schreieder

import time

R10matroid = matrix([[1,0,0,0,0,-1, 1, 0, 0, 1],
                     [0,1,0,0,0, 1,-1, 1, 0, 0],
                     [0,0,1,0,0, 0, 1,-1, 1, 0],
                     [0,0,0,1,0, 0, 0, 1,-1, 1],
                     [0,0,0,0,1, 1, 0, 0, 1,-1]])

K33cographic = matrix([[1,0,0,0,-1, 1, 0, 0, 1],
                       [0,1,0,0, 1,-1, 1, 0, 0],
                       [0,0,1,0, 0, 1,-1, 1, 0],
                       [0,0,0,1, 0, 0, 1,-1, 1]])

K33graphic = matrix([[1,0,0,0,0,-1, 1, 0, 0],
                     [0,1,0,0,0, 1,-1, 1, 0],
                     [0,0,1,0,0, 0, 1,-1, 1],
                     [0,0,0,1,0, 0, 0, 1,-1],
                     [0,0,0,0,1, 1, 0, 0, 1]])

Thetacographic = matrix([[1,0,1],
                         [0,1,1]])

K4graphic = matrix([[1,0,0, 0, 1, 1],
                    [0,1,0, 1, 0,-1],
                    [0,0,1,-1,-1, 0]])

K5graphic = matrix([[1,0,0,0, 1, 1, 1, 0, 0, 0],
                    [0,1,0,0,-1, 0, 0, 1, 1, 0],
                    [0,0,1,0, 0,-1, 0,-1, 0, 1],
                    [0,0,0,1, 0, 0,-1, 0,-1,-1]])

K6graphic = matrix([[1,0,0,0,0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [0,1,0,0,0,-1, 0, 0, 0, 1, 1, 1, 0, 0, 0],
                    [0,0,1,0,0, 0,-1, 0, 0,-1, 0, 0, 1, 1, 0],
                    [0,0,0,1,0, 0, 0,-1, 0, 0,-1, 0,-1, 0, 1],
                    [0,0,0,0,1, 0, 0, 0,-1, 0, 0,-1, 0,-1,-1]])

def r_value(matroid):
    return matroid.dimensions()[0]

def m_value(matroid):
    return matroid.dimensions()[1]

def matroid_dual(matroid):
    m = m_value(matroid)
    r = r_value(matroid)
    P = matroid.matrix_from_rows_and_columns(range(r), range(r, m))
    Q = P.transpose()
    Id = matrix.identity(m-r)
    return Q.augment(Id)

def to_base_p(n, p):
    if n == 0:
        return [0]
    digits = []
    while n > 0:
        digits.append(n % p)
        n //= p
    return digits[::-1]

def pad_zeros(l, length):
    return [0]*(length-len(l))+l if len(l) < length else l

def alb_vertices(p, matroid):
    m = m_value(matroid)
    r = r_value(matroid)
    points = []
    for i in range(p**(m-r)):
        point = pad_zeros(to_base_p(i, p), m-r)
        point = vector(point)
        points.append(point)
    return points

def alb_edges(p, matroid, reduced):
    if p!=2 and reduced == True:
        print('You cannot take the reduced graph unless your prime is 2!')
        return
    m = m_value(matroid)
    r = r_value(matroid)
    vertices = alb_vertices(p, matroid)
    vertex_index_map = {tuple(v): i for i, v in enumerate(vertices)}
    dual = matroid_dual(matroid)
    alb_edges = []
    for i in range(m):
        column = list(dual.column(i))
        if reduced == True:
            nonzero_index = [abs(x) for x in column].index(1)
        column = vector(column)
        for j in vertices:
            if p==2 and reduced == True:
                if j[nonzero_index] == 1:
                    continue
            if i<r:
                alb_edges.append([[j, (j - column) %p], i])
            else:
                alb_edges.append([[j, (j + column) %p], i])
    return alb_edges, vertex_index_map

def alb_matrix(p, matroid, reduced):
    m = m_value(matroid)
    r = r_value(matroid)
    vertices = alb_vertices(p, matroid)
    if reduced == False:
        dim = p**(m-r)
        dim2 = dim
    if reduced == True:
        dim = p**(m-r)//2
        dim2 = 2*dim
    alb_matrix = matrix(GF(p), dim2*r, dim*m, sparse=True)
    edges, vertex_index_map = alb_edges(p, matroid, reduced)
    for i in range(dim*m):
        color = edges[i][1]
        edge_start, edge_end = edges[i][0]
        start_index = vertex_index_map[tuple(edge_start)]
        end_index = vertex_index_map[tuple(edge_end)]
        if color < r:
            alb_matrix[dim2*color + start_index, i] -= 1
            alb_matrix[dim2*color + end_index, i] += 1
        else:
            contrib = matroid.column(color)
            for c in range(r):
                if contrib[c] != 0:
                    alb_matrix[dim2*c + start_index, i] -= contrib[c]
                    alb_matrix[dim2*c + end_index, i] += contrib[c]
    return alb_matrix

def alb_matrix_color(p, matroid, reduced):
    starting_matrix = alb_matrix(p, matroid, reduced)
    edges, vertex_index_map = alb_edges(p, matroid, reduced)
    m = m_value(matroid)
    r = r_value(matroid)
    if reduced == False:
        dim = p**(m-r)
        dim2 = dim
    if reduced == True:
        dim = p**(m-r)//2
        dim2 = 2*dim
    Z = matrix(GF(p), m, dim*m, sparse=True)
    next_matrix = starting_matrix.stack(Z)
    for i in range(dim*m):
        color = edges[i][1]
        next_matrix[dim2*r + color, i] += 1
    return next_matrix

def test_divisibility(p, matroid, reduced, name):
    start_time = time.time()
    print('Test matroid:', name,'at prime',p)
    mat = alb_matrix(p, matroid, reduced)
    rank = mat.rank()
    print('Matrix computing solutions:', mat.dimensions())
    print('The rank is', rank)
    mat_color = alb_matrix_color(p, matroid, reduced)
    rank_color = mat_color.rank()
    print('Matrix computing colorless solutions:', mat_color.dimensions())
    print('The colorless rank is', rank_color)
    if rank_color - rank == 0:
        print('It worked!')
        print(matroid)
        print('is obstructed mod',p)
        print('Both solution spaces have dimension', mat.ncols()-rank)
        print('Reduced has been toggled to:',reduced)
    if rank_color - rank != 0:
        print('It did not work.')
        print(matroid)
        print('is not obstructed mod',p)
        print('Solutions have dimension', mat.ncols()-rank)
        print('Colorless solutions have dimension', mat.ncols()-rank_color)
        print('Reduced has been toggled to:',reduced)
    end_time = time.time()
    print('Run time was',end_time - start_time,'seconds')
    print('')
    print('')


test_divisibility(2, K33graphic, True, 'K33graphic')
test_divisibility(2, K5graphic, True, 'K5graphic')
test_divisibility(2, K4graphic, True, 'K4graphic')
test_divisibility(2, R10matroid, True, 'R10matroid')
test_divisibility(2, K33cographic, True, 'K33cographic')
test_divisibility(2, Thetacographic, True, 'Thetacographic')
test_divisibility(2, K6graphic, True, 'K6graphic')

test_divisibility(3, K33graphic, False, 'K33graphic')
test_divisibility(3, K5graphic, False, 'K5graphic')
test_divisibility(3, R10matroid, False, 'R10matroid')

Test matroid: K33graphic at prime 2


Matrix computing solutions: (80, 72)
The rank is 57
Matrix computing colorless solutions: (89, 72)
The colorless rank is 57
It worked!
[ 1  0  0  0  0 -1  1  0  0]
[ 0  1  0  0  0  1 -1  1  0]
[ 0  0  1  0  0  0  1 -1  1]
[ 0  0  0  1  0  0  0  1 -1]
[ 0  0  0  0  1  1  0  0  1]
is obstructed mod 2
Both solution spaces have dimension 15
Reduced has been toggled to: True
Run time was 0.6439347267150879 seconds


Test matroid: K5graphic at prime 2
Matrix computing solutions: (256, 320)
The rank is 217
Matrix computing colorless solutions: (266, 320)
The colorless rank is 217
It worked!
[ 1  0  0  0  1  1  1  0  0  0]
[ 0  1  0  0 -1  0  0  1  1  0]
[ 0  0  1  0  0 -1  0 -1  0  1]
[ 0  0  0  1  0  0 -1  0 -1 -1]
is obstructed mod 2
Both solution spaces have dimension 103
Reduced has been toggled to: True
Run time was 0.10078024864196777 seconds


Test matroid: K4graphic at prime 2


Matrix computing solutions: (24, 24)
The rank is 17
Matrix computing colorless solutions: (30, 24)
The colorless rank is 18
It did not work.
[ 1  0  0  0  1  1]
[ 0  1  0  1  0 -1]
[ 0  0  1 -1 -1  0]
is not obstructed mod 2
Solutions have dimension 7
Colorless solutions have dimension 6
Reduced has been toggled to: True
Run time was 0.1626420021057129 seconds


Test matroid: R10matroid at prime 2
Matrix computing solutions: (160, 160)
The rank is 125
Matrix computing colorless solutions: (170, 160)
The colorless rank is 125
It worked!
[ 1  0  0  0  0 -1  1  0  0  1]
[ 0  1  0  0  0  1 -1  1  0  0]
[ 0  0  1  0  0  0  1 -1  1  0]
[ 0  0  0  1  0  0  0  1 -1  1]
[ 0  0  0  0  1  1  0  0  1 -1]
is obstructed mod 2
Both solution spaces have dimension 35
Reduced has been toggled to: True
Run time was 0.09552001953125 seconds


Test matroid: K33cographic at prime 2
Matrix computing solutions: (128, 144)
The rank is 103
Matrix computing colorless solutions: (137, 144)
The colorless rank is 1

Matrix computing solutions: (5120, 7680)
The rank is 4727


Matrix computing colorless solutions: (5135, 7680)
The colorless rank is 4727
It worked!
[ 1  0  0  0  0  1  1  1  1  0  0  0  0  0  0]
[ 0  1  0  0  0 -1  0  0  0  1  1  1  0  0  0]
[ 0  0  1  0  0  0 -1  0  0 -1  0  0  1  1  0]
[ 0  0  0  1  0  0  0 -1  0  0 -1  0 -1  0  1]
[ 0  0  0  0  1  0  0  0 -1  0  0 -1  0 -1 -1]
is obstructed mod 2
Both solution spaces have dimension 2953
Reduced has been toggled to: True
Run time was 3.0168356895446777 seconds


Test matroid: K33graphic at prime 3
Matrix computing solutions: (405, 729)
The rank is 352


Matrix computing colorless solutions: (414, 729)
The colorless rank is 353
It did not work.
[ 1  0  0  0  0 -1  1  0  0]
[ 0  1  0  0  0  1 -1  1  0]
[ 0  0  1  0  0  0  1 -1  1]
[ 0  0  0  1  0  0  0  1 -1]
[ 0  0  0  0  1  1  0  0  1]
is not obstructed mod 3
Solutions have dimension 377
Colorless solutions have dimension 376
Reduced has been toggled to: False
Run time was 0.2613956928253174 seconds


Test matroid: K5graphic at prime 3


Matrix computing solutions: (2916, 7290)
The rank is 2782


Matrix computing colorless solutions: (2926, 7290)
The colorless rank is 2783
It did not work.
[ 1  0  0  0  1  1  1  0  0  0]
[ 0  1  0  0 -1  0  0  1  1  0]
[ 0  0  1  0  0 -1  0 -1  0  1]
[ 0  0  0  1  0  0 -1  0 -1 -1]
is not obstructed mod 3
Solutions have dimension 4508
Colorless solutions have dimension 4507
Reduced has been toggled to: False
Run time was 1.5021092891693115 seconds


Test matroid: R10matroid at prime 3
Matrix computing solutions: (1215, 2430)
The rank is 1120


Matrix computing colorless solutions: (1225, 2430)
The colorless rank is 1121
It did not work.
[ 1  0  0  0  0 -1  1  0  0  1]
[ 0  1  0  0  0  1 -1  1  0  0]
[ 0  0  1  0  0  0  1 -1  1  0]
[ 0  0  0  1  0  0  0  1 -1  1]
[ 0  0  0  0  1  1  0  0  1 -1]
is not obstructed mod 3
Solutions have dimension 1310
Colorless solutions have dimension 1309
Reduced has been toggled to: False
Run time was 0.7169384956359863 seconds


